## Discussion Week 7: PageRank algorithm

The goal this week to understand the [PageRank algorithm](https://en.wikipedia.org/wiki/PageRank#:~:text=PageRank%20(PR)%20is%20an%20algorithm,the%20importance%20of%20website%20pages.) for HW 4.

### Problem description

- Think of a hyperlinked set of documents. Wikepedia is a good example.
- PageRank score: What is the probability that an internet surfer arrives at each document?
- Surfing rules:
    - The surfer starts at any document chosen at random.
    - The surfer can either move to the linked document with probability $d$, or jump to any random document (without following the link) with probability $1-d$. The probability $d$ is called a *damping factor*. 
    - The surfer moves to the next document that is linked to the current document.
    - If the current document has no link to other documents (this is called a *dangling node*), then the surfers moves to any document at random. (In other words, document with no outbound link is considered a document with outbound to all documents.)
- PageRank calculation rules:
    - PageRank is transferred from a given document to its outbound documents, divided equally among all outbound links.
    
**Remarks**
- Link-to-everything for dangling nodes: Dangling nodes will only receive PageRank without transfering.
- Damping factor: to allow traveling to all nodes (think of unconnected graphs).

### Mathematical formulation

- $N$: number of total documents
- $p_i$: $i$th document, $i = 1, \dots, N$
- $M(p_i)$: set of documents that link to $p_i$ (dangling node adjusted)
- $L(p_i)$: number of outbound links on $p_i$ (dangling node adjusted)


Then the *PageRank score* of $p_i$ is defined as
$$
PR(p_i) = \frac{1-d}{N} + d \sum_{p_j \in M(p_i)} \frac{PR(p_j)}{L(p_j)}.
$$

#### Matrix notation

- Adjacency matrix $A = [A_{ij}]$, $i,j = 1, \dots, N$, $A_{ij} = 1$ if $p_j$ has a link to $p_i$
- Scaled adjacency matrix $S = [S_{ij}]$, $i,j = 1, \dots, N$, $S_{ij} = A_{ij} / \sum_{i=1}^N A_{ij}$; scaled so that column sum is 1
- Transition matrix $H$: Replace the 0-column in $S$ with $[1/N, 1/N, \dots, 1/N]$. Formally,
$$
H_{ij} = \begin{cases}
S_{ij} & \text{if } \sum_{i=1}^N S_{ij} = 1 \\
1/N & \text{if } \sum_{i=1}^N S_{ij} = 0
\end{cases}
$$
- Google matrix $G$: With the damping factor $d$,
$$
G = dH + \frac{1-d}{N} J,
$$
where $J = [J_{ij}]$ where $J_{ij} = 1$ for all $i, j = 1, \dots, N$ (in other words, a $N\times N$ matrix of 1).
- Finally, the PageRank score vector $v$ is the solution of
$$
v = Gv.
$$
- In other words, $v$ is the 1st right eigenvector of $G$.

**Construction of $G$**
\begin{align*}
v_i &= \frac{1-d}{N} + d \sum_{j \in M(i)} \frac{v_j}{L(p_j)} \\
&= \frac{1-d}{N} + d v^\top H_j, \; H_j: j \text{th column of } H
\end{align*}

Stacking $v_i$, $i=1, \dots, N$ into a column vector $v$,
\begin{align*}
v &= \frac{1-d}{N} \mathbf{1}+ d Hv, \; \mathbf{1}: \text{column vector of ones} = (1, 1, \dots, 1)^\top \\
&= \left( \frac{1-d}{N} J+ d H \right)v \\
&= Gv
\end{align*}

The second equality is because $J = \mathbf{1} \mathbf{1}^\top$ and $\mathbf{1}^\top v = 1$.

**Remarks**
- [More about the last point](https://en.wikipedia.org/wiki/Perron%E2%80%93Frobenius_theorem): The first eigenvalue is 1, and the elements of the corresponding eigenvector is positive.
- Some people define adjacency matrix as the element being 1 when $p_i$ has a link to $p_j$. This is equivalent to considering the transpose of above matrices, and considering the left eigenvector.

### Calculation
- [Power iteration](https://en.wikipedia.org/wiki/Power_iteration): Knowing that the corresponding eigenvalue is 1, the algorithm looks like:
    1. Initialize $v^{(0)} = \left( \begin{matrix} 1/N \\ 1/N \\ \vdots \\ 1/N \end{matrix} \right)$.
    2. For $t = 0, 1, \dots$, update $v^{(t+1)} = G v^{(t)}$.

### Example

![A web with four documents.](graph.png)

Please note that the codes below are not optimal and only shows a rough idea.

In [1]:
# Example
import numpy as np
from scipy.sparse import csr_matrix

## Adjacency matrix
# A = np.matrix([[0,0,1,0], [1,0,1,0], [1,0,0,0], [0,0,1,0]])
N = 4
row = np.array([1, 2, 0, 1, 3])
col = np.array([0, 0, 2, 2, 2])
data = np.array([1/2, 1/2, 1/3, 1/3, 1/3])
A = csr_matrix((data, (row, col)), shape=(N, N)) # for sparse matrix
A.toarray()

array([[0.        , 0.        , 0.33333333, 0.        ],
       [0.5       , 0.        , 0.33333333, 0.        ],
       [0.5       , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.33333333, 0.        ]])

In [2]:
L = A.sum(axis = 0) # column sum: number of outbounds from each node
H = A
H[:, np.where(L.A1 == 0)[0]] = 1/N # dangling adjustment

/Users/soobin.kim/opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [3]:
H.toarray()

array([[0.        , 0.25      , 0.33333333, 0.25      ],
       [0.5       , 0.25      , 0.33333333, 0.25      ],
       [0.5       , 0.25      , 0.        , 0.25      ],
       [0.        , 0.25      , 0.33333333, 0.25      ]])

In [4]:
d = 0.85
G = d*H + (1-d) / N * np.ones(N*N).reshape(N, N)
G

matrix([[0.0375    , 0.25      , 0.32083333, 0.25      ],
        [0.4625    , 0.25      , 0.32083333, 0.25      ],
        [0.4625    , 0.25      , 0.0375    , 0.25      ],
        [0.0375    , 0.25      , 0.32083333, 0.25      ]])

In [5]:
v0 = np.array([1 / N] * N).reshape(4, 1)
for _ in range(10):
    v = G @ v0
    v0 = v

In [6]:
v

matrix([[0.22048846],
        [0.31419566],
        [0.24482742],
        [0.22048846]])

In [7]:
G @ v

matrix([[0.22048814],
        [0.31419574],
        [0.24482797],
        [0.22048814]])

In [8]:
# Compare with...
eee = np.linalg.eig(G)
ev1 = eee[1][:,0] 
ev1 / sum(ev1)

matrix([[0.22048822],
        [0.31419572],
        [0.24482783],
        [0.22048822]])

In [9]:
# Another way: uses sparse matrix H during calculation, 
## could be more efficient.
v0 = np.array([1 / N] * N).reshape(4, 1)
for _ in range(10):
    v = d * H@v + (1-d) * np.mean(v0)
    v0 = v

In [10]:
v

matrix([[0.22048822],
        [0.31419572],
        [0.24482783],
        [0.22048822]])